## Imports

In [1]:
import os
import data_loading as dl
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


## Part 1: Loading the Experiment(s) Data
Based on the data retrieved from `data_analysis.ipynb`, get that data into a format capable of being used by a Machine Learning model.

<span style="color: orange;">**Future Experiment:** Generating multiple months or simply estimating a year in advance.</span>

In [2]:
MODEL_DATA_DIRECTORY = 'model_data/wheat_model_data/'
N_TREND_SAMPLES = 3 # Number of price samples for "history" trend (training data)
N_MONTHS_AHEAD = 1 # Number of months ahead to predict
TEST_SIZE = 0.2
OUTPUT_COLUMN_NAME = 'Price'

# Get the number of features for this model (given the model_data chosen)
sample_file_path = os.path.join(MODEL_DATA_DIRECTORY, os.listdir(MODEL_DATA_DIRECTORY)[0])
sample_columns = pd.read_csv(sample_file_path).columns[1:] # Skip 'Unnamed: 0' column
num_input_samples = len(sample_columns[2:-1]) + N_TREND_SAMPLES + 1 # +1 for 'n_previous_prices' later
print("Num Features: ", num_input_samples)

Num Features:  9


In [3]:
model_data = dl.get_model_data_from_directory(MODEL_DATA_DIRECTORY, sample_columns, N_TREND_SAMPLES)

c:\GitHub\Predicting-Commodity-Food-Pricing\data_loading.py:175: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_data = pd.concat([model_data, data_to_concat], ignore_index=True).drop(columns=['Unnamed: 0'])


In [4]:
inputs_and_outputs = model_data.apply(dl.divide_inputs_and_outputs, axis=1)
inputs = inputs_and_outputs['inputs'].tolist(); outputs = inputs_and_outputs['output'].tolist()
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=TEST_SIZE, shuffle=False) # Don't shuffle, should be cohesive samples not seen
x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor, scaler = dl.format_for_ML_usage_torch(inputs, outputs, num_input_samples)

print('x_train_tensor shape:', x_train_tensor.shape)
print('x_test_tensor shape:', x_test_tensor.shape)
print('y_train_tensor shape:', y_train_tensor.shape)
print('y_test_tensor shape:', y_test_tensor.shape)

x_train_tensor = torch.tensor(x_train_tensor, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_tensor, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
test_data = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)


x_train_tensor shape: torch.Size([709, 9, 1])
x_test_tensor shape: torch.Size([178, 9, 1])
y_train_tensor shape: torch.Size([709, 1])
y_test_tensor shape: torch.Size([178, 1])


C:\Users\balbonis\AppData\Local\Temp\ipykernel_39984\1889222699.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_tensor = torch.tensor(x_train_tensor, dtype=torch.float32)
C:\Users\balbonis\AppData\Local\Temp\ipykernel_39984\1889222699.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)
C:\Users\balbonis\AppData\Local\Temp\ipykernel_39984\1889222699.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_tensor, dtype=torch.flo

## Part 2: Building Complex ML Models

In [5]:
def positional_encoding(length, depth):
    """
    Compute the positional encoding for a transformer model.
    :param length: The length of the input sequence
    :param depth: The depth of the model
    :return: A tensor containing the positional encoding for the input sequence
    """
    # Create a position array with shape [length, 1] and a div_term array with shape [1, depth]
    position = torch.arange(length).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, depth, 2) * -(torch.log(torch.tensor(10000.0)) / depth))
    
    # Compute the positional encoding
    pos_enc = torch.zeros(length, depth)
    pos_enc[:, 0::2] = torch.sin(position * div_term)
    pos_enc[:, 1::2] = torch.cos(position * div_term)
    
    return pos_enc

### Model 1: Simple, 1-Layer Transformer

In [22]:
# Model Architecture
HEAD_SIZE = 64
NUM_HEADS = 8
FF_DIM = 16

# ML Optimizer
LEARNING_RATE = 1e-5
CLIP_VALUE = 0.5 # Gradient Clipping (https://neptune.ai/blog/understanding-gradient-clipping-and-how-it-can-fix-exploding-gradients-problem)
DROP_OUT_RATE = 0.2

d_model = x_train_tensor.shape[-1] 
length = x_train_tensor.shape[1]  

In [7]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, head_size, num_heads, ff_dim, dropout=0):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout)
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=ff_dim, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=ff_dim, out_channels=d_model, kernel_size=1)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value):
        # Note: MultiheadAttention expects input of shape (L, N, E) where L is the sequence length, N is the batch size, and E is the embedding dimension.
        attn_output, _ = self.attention(query, key, value)
        out1 = self.layernorm1(query + attn_output)
        
        # Conv1D layers expect input of shape (N, C, L), hence we permute
        out1_permuted = out1.permute(1, 2, 0)
        ff_output = F.relu(self.conv1(out1_permuted))
        ff_output = self.conv2(ff_output)
        
        # Permute back to match the MultiheadAttention output shape
        ff_output = ff_output.permute(2, 0, 1)
        out2 = self.layernorm2(out1 + self.dropout(ff_output))
        
        return out2

In [8]:
class TransformerModel(nn.Module):
    def __init__(self, num_input_samples, d_model, head_size, num_heads, ff_dim, dropout=0, num_transformers=10):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.input_projection = nn.Linear(1, d_model)
        self.pos_encoding = positional_encoding(num_input_samples, d_model)
        self.transformers = nn.ModuleList([TransformerBlock(d_model, head_size, num_heads, ff_dim, dropout) for _ in range(num_transformers)])
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)
        self.output_layer = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_projection(x)
        pos_encoding = self.pos_encoding[:x.size(1), :].unsqueeze(0).expand_as(x)
        x += pos_encoding
        for transformer in self.transformers:
            x = transformer(x, x, x)
        x = x.permute(1, 2, 0)  # Rearrange dimensions for global pooling
        x = self.global_avg_pooling(x).squeeze(-1)
        x = self.output_layer(x)
        return x

In [23]:
# Define the model, loss function, and optimizer
model = TransformerModel(num_input_samples, d_model=d_model, head_size=HEAD_SIZE, num_heads=NUM_HEADS, ff_dim=FF_DIM, dropout=DROP_OUT_RATE)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

AssertionError: embed_dim must be divisible by num_heads

## Part 3: Training the Model
This section is specifically for training the model built in the previous section. Some contants (`NUM_EPOCHS`, `BATCH_SIZE`) are provided and should be the only required parameters to adjust for this section of the experiment. 

A plot of the loss throughout the training process is provided for easy understanding about if the model is overfitting or underfitting. For a review of these concepts, see [this article](https://www.analyticsfordecisions.com/overfitting-and-underfitting/#:~:text=Overfitting%20happens%20when%20the%20model%20is%20too%20complex,poor%20performance%20on%20both%20training%20and%20test%20datasets.).
<br/><br/>
**Potential Future Parameters**
* **Regularization:** L1, L2, Dropout; helps prevent overfitting

In [10]:
# def learning_rate_scheduler(epoch, lr):
#     """
#     Learning rate scheduler
#     :param int epoch: current epoch
#     :param float lr: current learning rate
#     """
#     if epoch < 15:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)
    
# callback = LearningRateScheduler(learning_rate_scheduler)

In [11]:
NUM_EPOCHS = 7
BATCH_SIZE = 3 # Fiscal Quarters

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)

In [16]:
print(train_data.tensors[0].reshape(-1, 9, ).shape)
print(test_data.tensors[0].reshape(-1, 9, ).shape)

torch.Size([709, 9])
torch.Size([178, 9])


In [19]:
def train(model, data_loader, optimizer, criterion):
    model.train()  # Set the model to training mode
    total_loss = 0.
    for batch, (input, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(input)
        print(output, "\n", target)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

# Example data loading and training loop
# Assuming data_loader is defined and loads your dataset
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train_loss = train(model, train_loader, optimizer, criterion)
    print(f'| epoch {epoch:3d} | train_loss {train_loss:5.2f} |')


tensor([[ 0.1695],
        [ 0.0192],
        [ 0.1026],
        [-0.0303],
        [ 0.0684],
        [ 0.1273],
        [ 0.1242],
        [ 0.3542],
        [-0.0020]], grad_fn=<AddmmBackward0>) 
 tensor([[0.1578],
        [0.2373],
        [0.1040]])


c:\GitHub\Predicting-Commodity-Food-Pricing\.venv\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (9) must match the size of tensor b (3) at non-singleton dimension 0

In [ ]:
# Plot the loss over time
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

## Part 4: Examining Results
In this section, we're plotting the model's predictions versus the actual price point for the commodity in question. However, one plot focuses specifically on the testing data only (this is a better plot to see how well the model is performing/generalizing) and the other focuses on the entire dataset (this is a better plot to see if the model is correlating to the provided dataset at all).

Therefore, when **evaluating the performance** of the model, **the first plot should be used.** 

In [ ]:
# Plot the model's predictions of the testing data vs the actual data
predictions = model.predict(x_test_tensor)
predictions = scaler.inverse_transform(predictions)
predictions = predictions.reshape(-1)

plt.plot(y_test, label='Actual')
plt.plot(predictions, label='Predicted')
plt.title('Model Predictions (Testing Data Only)')
plt.ylabel('Price')
plt.xlabel('Months')
plt.legend()
plt.show()

# We use Mean Absolute Error (MAE) as the final accuracy metric because it is the true error (difference) between the actual and predicted values
final_mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error (FINAL ACCURACY METRIC):', final_mae)

In [ ]:
# Plot the model's predictions (both training and testing) vs the actual data
predictions = model.predict(x_train_tensor)
predictions = scaler.inverse_transform(predictions)
predictions = predictions.reshape(-1)

plt.plot(y_train, label='Actual')
plt.plot(predictions, label='Predicted')
plt.title('Model Predictions (Training and Testing Data)')
plt.ylabel('Price')
plt.xlabel('Days')
plt.legend()
plt.show()

In [ ]:
# Save the model to saved_models
model.save(f'saved_models/wheat_price_transformer_model_{final_mae}.h5')